# DB year 컬럼 추가 스크립트

## 목적
- `any_approval.documents` 테이블에 `year` 컬럼 추가
- `created_at` (유닉스 타임스탬프 밀리초) → 한국시간 기준 연도 추출하여 저장
- **year 컬럼만 추가/수정, 다른 컬럼은 절대 변경 없음**

In [2]:
import pymysql
from datetime import datetime, timezone, timedelta

## 0. 설정

In [3]:
# ========== DB 설정 ==========
DB_CONFIG = {
    'host': 'localhost',
    'port': 3306,
    'user': 'root',
    'password': '1234',  # 비밀번호 입력
    'database': 'any_approval',
    'charset': 'utf8mb4'
}

# 한국 시간대
KST = timezone(timedelta(hours=9))

print("DB 설정 완료")

DB 설정 완료


## 1. 현재 상태 확인

In [4]:
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

# 현재 테이블 구조 확인
cursor.execute("DESCRIBE documents")
columns = cursor.fetchall()

print("현재 테이블 구조:")
column_names = []
year_exists = False
for col in columns:
    col_name = col[0]
    column_names.append(col_name)
    print(f"  {col[0]}: {col[1]}")
    if col_name == 'year':
        year_exists = True

print(f"\nyear 컬럼 존재 여부: {year_exists}")

# 현재 행 수
cursor.execute("SELECT COUNT(*) FROM documents")
total_rows = cursor.fetchone()[0]
print(f"총 행 수: {total_rows:,}개")

cursor.close()
conn.close()

현재 테이블 구조:
  id: int(11)
  source_id: varchar(50)
  doc_num: varchar(100)
  doc_type: varchar(50)
  title: varchar(500)
  doc_status: varchar(50)
  created_at: bigint(20)
  drafter_name: varchar(100)
  drafter_position: varchar(100)
  drafter_dept: varchar(100)
  drafter_email: varchar(100)
  drafter_dept_code: varchar(50)
  form_name: varchar(200)
  is_public: tinyint(1)
  end_year: int(11)
  references: text
  attaches: text
  referrers: text
  activities: text
  doc_body: mediumtext
  created_date: timestamp

year 컬럼 존재 여부: False
총 행 수: 23,320개


## 2. 변경 전 데이터 스냅샷 (검증용)

In [5]:
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

# 변경 전 데이터 스냅샷 (id, source_id, created_at, doc_num 등 주요 컬럼)
cursor.execute("""
    SELECT id, source_id, created_at, doc_num, doc_type, title, doc_status, drafter_name
    FROM documents
    ORDER BY id
""")
before_snapshot = cursor.fetchall()

print(f"스냅샷 저장 완료: {len(before_snapshot):,}개 행")

# 샘플 확인
print("\n샘플 (처음 3개):")
for row in before_snapshot[:3]:
    print(f"  id={row[0]}, source_id={row[1]}, created_at={row[2]}, doc_num={row[3]}")

cursor.close()
conn.close()

스냅샷 저장 완료: 23,320개 행

샘플 (처음 3개):
  id=1, source_id=15167823, created_at=1609740886000, doc_num=ANY-사업-791
  id=2, source_id=15168441, created_at=1609743056000, doc_num=ANY-사업수행-482
  id=3, source_id=15162004, created_at=1609717361000, doc_num=ANY-경비627


## 3. year 컬럼 추가 (없는 경우에만)

In [6]:
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

if not year_exists:
    print("year 컬럼 추가 중...")
    cursor.execute("ALTER TABLE documents ADD COLUMN year INT NULL")
    conn.commit()
    print("✅ year 컬럼 추가 완료")
else:
    print("year 컬럼이 이미 존재합니다.")

# 확인
cursor.execute("DESCRIBE documents")
columns_after = cursor.fetchall()
print("\n추가 후 테이블 구조:")
for col in columns_after:
    print(f"  {col[0]}: {col[1]}")

cursor.close()
conn.close()

year 컬럼 추가 중...
✅ year 컬럼 추가 완료

추가 후 테이블 구조:
  id: int(11)
  source_id: varchar(50)
  doc_num: varchar(100)
  doc_type: varchar(50)
  title: varchar(500)
  doc_status: varchar(50)
  created_at: bigint(20)
  drafter_name: varchar(100)
  drafter_position: varchar(100)
  drafter_dept: varchar(100)
  drafter_email: varchar(100)
  drafter_dept_code: varchar(50)
  form_name: varchar(200)
  is_public: tinyint(1)
  end_year: int(11)
  references: text
  attaches: text
  referrers: text
  activities: text
  doc_body: mediumtext
  created_date: timestamp
  year: int(11)


## 4. 연도 계산 함수

In [7]:
def timestamp_to_year_kst(timestamp_ms):
    """
    밀리초 타임스탬프 → 한국시간 기준 연도
    """
    if timestamp_ms is None:
        return None
    dt = datetime.fromtimestamp(timestamp_ms / 1000, tz=KST)
    return dt.year

def timestamp_to_kst_str(timestamp_ms):
    """
    밀리초 타임스탬프 → 한국시간 문자열 (확인용)
    """
    if timestamp_ms is None:
        return None
    dt = datetime.fromtimestamp(timestamp_ms / 1000, tz=KST)
    return dt.strftime('%Y-%m-%d %H:%M:%S')

# 테스트
test_ts = 1422345818000
print(f"테스트: {test_ts}")
print(f"  한국시간: {timestamp_to_kst_str(test_ts)}")
print(f"  연도: {timestamp_to_year_kst(test_ts)}")

테스트: 1422345818000
  한국시간: 2015-01-27 17:03:38
  연도: 2015


## 5. year 값 UPDATE

In [8]:
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

# 모든 행의 id, created_at 조회
cursor.execute("SELECT id, created_at FROM documents")
rows = cursor.fetchall()

print(f"UPDATE 대상: {len(rows):,}개 행")

# UPDATE 실행
update_count = 0
error_count = 0
year_distribution = {}  # 연도별 분포 확인용

for row_id, created_at in rows:
    year = timestamp_to_year_kst(created_at)
    
    if year is None:
        error_count += 1
        continue
    
    # year 컬럼만 UPDATE
    cursor.execute("UPDATE documents SET year = %s WHERE id = %s", (year, row_id))
    update_count += 1
    
    # 분포 기록
    year_distribution[year] = year_distribution.get(year, 0) + 1

conn.commit()

print(f"\nUPDATE 완료: {update_count:,}개")
print(f"에러 (created_at이 NULL): {error_count}개")

print(f"\n연도별 분포:")
for year in sorted(year_distribution.keys()):
    print(f"  {year}년: {year_distribution[year]:,}개")

cursor.close()
conn.close()

UPDATE 대상: 23,320개 행

UPDATE 완료: 23,320개
에러 (created_at이 NULL): 0개

연도별 분포:
  2010년: 841개
  2011년: 855개
  2012년: 1,280개
  2013년: 1,582개
  2014년: 1,504개
  2015년: 1,367개
  2016년: 1,528개
  2017년: 1,963개
  2018년: 1,661개
  2019년: 1,478개
  2020년: 1,075개
  2021년: 1,331개
  2022년: 1,818개
  2023년: 1,442개
  2024년: 1,811개
  2025년: 1,784개


## 6. 검증

In [9]:
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

print("=" * 60)
print("검증 1: 다른 컬럼 변경 여부 확인")
print("=" * 60)

# 변경 후 데이터 조회 (year 제외한 동일 컬럼)
cursor.execute("""
    SELECT id, source_id, created_at, doc_num, doc_type, title, doc_status, drafter_name
    FROM documents
    ORDER BY id
""")
after_snapshot = cursor.fetchall()

# 비교
mismatch_count = 0
for before, after in zip(before_snapshot, after_snapshot):
    if before != after:
        mismatch_count += 1
        print(f"  불일치 발견! id={before[0]}")
        print(f"    변경 전: {before}")
        print(f"    변경 후: {after}")
        if mismatch_count >= 5:
            print("  ... (더 있음)")
            break

if mismatch_count == 0:
    print("✅ 모든 기존 컬럼 값 동일 (year만 추가됨)")
else:
    print(f"❌ 불일치 발견: {mismatch_count}건")

cursor.close()
conn.close()

검증 1: 다른 컬럼 변경 여부 확인
✅ 모든 기존 컬럼 값 동일 (year만 추가됨)


In [10]:
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

print("=" * 60)
print("검증 2: year 값 NULL 여부 확인")
print("=" * 60)

cursor.execute("SELECT COUNT(*) FROM documents WHERE year IS NULL")
null_count = cursor.fetchone()[0]

if null_count == 0:
    print("✅ 모든 행에 year 값이 있음")
else:
    print(f"❌ year가 NULL인 행: {null_count}개")
    
    # NULL인 행 확인
    cursor.execute("SELECT id, source_id, created_at FROM documents WHERE year IS NULL LIMIT 5")
    null_rows = cursor.fetchall()
    for row in null_rows:
        print(f"  id={row[0]}, source_id={row[1]}, created_at={row[2]}")

cursor.close()
conn.close()

검증 2: year 값 NULL 여부 확인
✅ 모든 행에 year 값이 있음


In [11]:
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

print("=" * 60)
print("검증 3: year 값 정확성 확인 (샘플)")
print("=" * 60)

# 랜덤 10개 샘플링
cursor.execute("""
    SELECT id, source_id, created_at, year 
    FROM documents 
    ORDER BY RAND() 
    LIMIT 10
""")
samples = cursor.fetchall()

all_correct = True
for row_id, source_id, created_at, db_year in samples:
    calculated_year = timestamp_to_year_kst(created_at)
    kst_str = timestamp_to_kst_str(created_at)
    
    status = "✅" if db_year == calculated_year else "❌"
    if db_year != calculated_year:
        all_correct = False
    
    print(f"{status} id={row_id}, source_id={source_id}")
    print(f"   created_at: {created_at} → KST: {kst_str}")
    print(f"   DB year: {db_year}, 계산 year: {calculated_year}")

if all_correct:
    print("\n✅ 모든 샘플 정확")
else:
    print("\n❌ 불일치 발견")

cursor.close()
conn.close()

검증 3: year 값 정확성 확인 (샘플)
✅ id=6668, source_id=24771796
   created_at: 1739927077000 → KST: 2025-02-19 10:04:37
   DB year: 2025, 계산 year: 2025
✅ id=13757, source_id=11668085
   created_at: 1556611268000 → KST: 2019-04-30 17:01:08
   DB year: 2019, 계산 year: 2019
✅ id=18848, source_id=2005873
   created_at: 1374653082000 → KST: 2013-07-24 17:04:42
   DB year: 2013, 계산 year: 2013
✅ id=6238, source_id=24300003
   created_at: 1733444139000 → KST: 2024-12-06 09:15:39
   DB year: 2024, 계산 year: 2024
✅ id=10094, source_id=7159121
   created_at: 1489539645000 → KST: 2017-03-15 10:00:45
   DB year: 2017, 계산 year: 2017
✅ id=5622, source_id=23481003
   created_at: 1722476591000 → KST: 2024-08-01 10:43:11
   DB year: 2024, 계산 year: 2024
✅ id=22070, source_id=2009105
   created_at: 1447661948000 → KST: 2015-11-16 17:19:08
   DB year: 2015, 계산 year: 2015
✅ id=20106, source_id=2007138
   created_at: 1399548379000 → KST: 2014-05-08 20:26:19
   DB year: 2014, 계산 year: 2014
✅ id=12136, source_id=9400397


In [12]:
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

print("=" * 60)
print("검증 4: 행 수 변경 여부 확인")
print("=" * 60)

cursor.execute("SELECT COUNT(*) FROM documents")
after_total = cursor.fetchone()[0]

print(f"변경 전 행 수: {total_rows:,}개")
print(f"변경 후 행 수: {after_total:,}개")

if total_rows == after_total:
    print("✅ 행 수 동일")
else:
    print("❌ 행 수 변경됨!")

cursor.close()
conn.close()

검증 4: 행 수 변경 여부 확인
변경 전 행 수: 23,320개
변경 후 행 수: 23,320개
✅ 행 수 동일


In [14]:
print("=" * 60)
print("검증 5: cmds 파일과 비교 (선택사항)")
print("=" * 60)

# cmds 연도별 파일이 있다면 비교 가능
import os
from glob import glob
import re

BASE_DIR = r'C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관'
YEARLY_DIR = os.path.join(BASE_DIR, 'yearly_final_ZB00')

if os.path.exists(YEARLY_DIR):
    # cmds에서 source_id와 연도 매핑
    cmds_year_map = {}  # source_id_num → year (from filename)
    
    for cmds_file in glob(os.path.join(YEARLY_DIR, 'documents_*.cmds')):
        filename = os.path.basename(cmds_file)
        # documents_2015.cmds → 2015
        year_match = re.search(r'documents_(\d{4})\.cmds', filename)
        if year_match:
            file_year = int(year_match.group(1))
            
            with open(cmds_file, 'r', encoding='utf-8') as f:
                for line in f:
                    source_match = re.search(r'"sourceId":"doc_(\d+)_', line)
                    if source_match:
                        source_id_num = source_match.group(1)
                        cmds_year_map[source_id_num] = file_year
    
    print(f"cmds에서 추출한 문서 수: {len(cmds_year_map):,}개")
    
    # DB와 비교
    conn = pymysql.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("SELECT source_id, year FROM documents")
    db_rows = cursor.fetchall()
    cursor.close()
    conn.close()
    
    mismatch = 0
    matched = 0
    for source_id, db_year in db_rows:
        source_id_str = str(source_id)
        if source_id_str in cmds_year_map:
            cmds_year = cmds_year_map[source_id_str]
            if db_year == cmds_year:
                matched += 1
            else:
                mismatch += 1
                if mismatch <= 5:
                    print(f"  불일치: source_id={source_id}, DB year={db_year}, cmds year={cmds_year}")
    
    print(f"\n매칭 결과:")
    print(f"  일치: {matched:,}개")
    print(f"  불일치: {mismatch}개")
    
    if mismatch == 0:
        print("✅ cmds 파일과 DB year 값 일치")
else:
    print(f"cmds 폴더 없음: {YEARLY_DIR}")
    print("이 검증은 건너뜁니다.")

검증 5: cmds 파일과 비교 (선택사항)
cmds에서 추출한 문서 수: 23,320개

매칭 결과:
  일치: 23,320개
  불일치: 0개
✅ cmds 파일과 DB year 값 일치


## 완료!

### 수행 작업
- `documents` 테이블에 `year` INT 컬럼 추가
- `created_at` → 한국시간 기준 연도 계산하여 `year`에 저장

### 검증 항목
1. 다른 컬럼 값 변경 없음
2. year 값 NULL 없음
3. year 값 정확성 (샘플 확인)
4. 행 수 동일
5. cmds 파일과 비교 (선택)